In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SynapseToGoldLayer") \
    .config("spark.jars.packages", "com.microsoft.azure:synapse-spark-connector:2.1.0") \
    .getOrCreate()
    
spark.conf.set(f"fs.azure.account.key.{account_name}.dfs.core.windows.net", account_key)

In [0]:
synapse_server= "aviation-synapse-ws-ondemand.sql.azuresynapse.net"
synapse_database= "Aviation"
synapse_username= "adminuser"
synapse_password= "SecureP@ssword123"

output_base_path = "abfss://gold@aviationdatalake.dfs.core.windows.net/"

In [0]:
def dump_to_gold(view, output_path):
    synapse_options = {
        "url": f"jdbc:sqlserver://{synapse_server}:1433;database={synapse_database}",
        "dbtable": f"{view}",  # Replace with your view name
        "user": f"{synapse_username}",
        "password": f"{synapse_password}",
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }

    df = spark.read.format("jdbc").options(**synapse_options).load()

    df.write.format("delta").option("mergeSchema", "true").mode("append").save(f"{output_path}")

    print("Data successfully written to Gold layer.")


In [0]:
#cities
dump_to_gold(
    view = "gold.dim_cities",
    output_path = f"{output_base_path}/dim_cities/"
)
#countries
dump_to_gold(
    view = "gold.dim_countries",
    output_path = f"{output_base_path}/dim_countries/"
)
#airplanes
dump_to_gold(
    view = "gold.dim_airplanes",
    output_path = f"{output_base_path}/dim_airplanes/"
)
#airlines
dump_to_gold(
    view = "gold.dim_airlines",
    output_path = f"{output_base_path}/dim_airlines/"
)
#airports
dump_to_gold(
    view = "gold.dim_airports",
    output_path = f"{output_base_path}/dim_airports/" 
)
#flights
dump_to_gold(
    view = "gold.fact_flights",
    output_path = f"{output_base_path}/fact_flights/" 
)

Data successfully written to Gold layer.
Data successfully written to Gold layer.
Data successfully written to Gold layer.
Data successfully written to Gold layer.
Data successfully written to Gold layer.
Data successfully written to Gold layer.
